In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training.json
/kaggle/input/exist2025-all/EXIST2025_dev_en.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_EASE_es.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_majority_class_hard.json
/kaggle/input/exist2025-all/EXIST2025_training_translated_en.json
/kaggle/input/exist2025-all/EXIST2025_training_translated_es.json
/kaggle/input/exist2025-all/EXIST2025_dev_es.json
/kaggle/input/exist2025-all/EXIST2025_training_task1_3_minority_class_soft.json
/kaggle/input/exist2025-all/EXIST2025_training_EASE_en.json
/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_m

In [4]:
import wandb

wandb.login(key="0c5f368f1f51fd942ec7bb3a1c74efb7bdc832d6")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [10]:
# import json
# import torch
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MultiLabelBinarizer
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# from torch.utils.data import Dataset

# # Load English and Spanish datasets separately
# with open("/kaggle/input/exist2025-all/EXIST2025_training_translated_en.json", "r", encoding="utf-8") as f:
#     data_en = json.load(f)

# with open("/kaggle/input/exist2025-all/EXIST2025_training_translated_es.json", "r", encoding="utf-8") as f:
#     data_es = json.load(f)

# Define correct label classes
CORRECT_LABELS = [
    "IDEOLOGICAL-INEQUALITY",
    "MISOGYNY-NON-SEXUAL-VIOLENCE",
    "OBJECTIFICATION",
    "SEXUAL-VIOLENCE",
    "STEREOTYPING-DOMINANCE",
    "NO"  # Represents non-sexist tweets (previously "-")
]

# # Function to process dataset
# def process_data(data):
#     tweets = []
#     labels = []
#     ids = []

#     for entry in data.values():
#         tweet_id = entry["id_EXIST"]
#         tweet = entry["tweet"]
#         is_sexist = any(label == "YES" for label in entry["labels_task1_1"])  # Check if at least one annotator marked it sexist
#         label = entry["labels_task1_3"] if is_sexist else [["NO"]]  # Non-sexist tweets get "NO"

#         # Flatten labels
#         processed_labels = [l if l != "-" else "NO" for sublist in label for l in sublist]

#         # Remove "UNKNOWN"
#         processed_labels = [l for l in processed_labels if l != "UNKNOWN"]

#         # Ensure every tweet has at least one label
#         if not processed_labels:
#             processed_labels = ["NO"]

#         tweets.append(tweet)
#         labels.append(processed_labels)
#         ids.append(tweet_id)

#     return tweets, labels, ids

# # Process data for English and Spanish
# english_tweets, english_labels, english_ids = process_data(data_en)
# spanish_tweets, spanish_labels, spanish_ids = process_data(data_es)

# # MultiLabel Binarizer with Fixed Labels
# mlb = MultiLabelBinarizer(classes=CORRECT_LABELS)  # Force correct label order
# english_labels_bin = mlb.fit_transform(english_labels)
# spanish_labels_bin = mlb.transform(spanish_labels)  # Use the same binarizer

# label_classes = mlb.classes_
# print(f"Corrected Label Classes: {label_classes}")  # Debugging

# # Tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Custom Dataset Class
# class TweetDataset(Dataset):
#     def __init__(self, texts, labels, ids, tokenizer, max_length=256):
#         self.texts = texts
#         self.labels = labels
#         self.ids = ids
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         text = self.texts[idx]
#         tweet_id = self.ids[idx]
#         labels = torch.tensor(self.labels[idx], dtype=torch.float)
#         encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

#         return {
#             "id": tweet_id,
#             "input_ids": encoding["input_ids"].squeeze(0),
#             "attention_mask": encoding["attention_mask"].squeeze(0),
#             "labels": labels
#         }

# # **Train-validation split**
# def get_datasets(tweets, labels, ids):
#     train_texts, val_texts, train_labels, vallabels, train_ids, val_ids = train_test_split(
#         tweets, labels, ids, test_size=0.2, random_state=42
#     )
#     train_dataset = TweetDataset(train_texts, train_labels, train_ids, tokenizer)
#     val_dataset = TweetDataset(val_texts, val_labels, val_ids, tokenizer)
#     return train_dataset, val_dataset

# # Split English and Spanish datasets
# train_dataset_en, val_dataset_en = get_datasets(english_tweets, english_labels_bin, english_ids)
# train_dataset_es, val_dataset_es = get_datasets(spanish_tweets, spanish_labels_bin, spanish_ids)

# # Define and Train Model for English
# model_en = BertForSequenceClassification.from_pretrained(
#     "bert-base-multilingual-cased", 
#     num_labels=len(label_classes), 
#     problem_type="multi_label_classification"
# )

# training_args_en = TrainingArguments(
#     output_dir="./results/en",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     num_train_epochs=4,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss"
# )

# trainer_en = Trainer(
#     model=model_en,
#     args=training_args_en,
#     train_dataset=train_dataset_en,  
#     eval_dataset=val_dataset_en  # Now includes validation set
# )

# # Train English Model
# trainer_en.train()

# # Define and Train Model for Spanish
# model_es = BertForSequenceClassification.from_pretrained(
#     "bert-base-multilingual-cased", 
#     num_labels=len(label_classes), 
#     problem_type="multi_label_classification"
# )

# training_args_es = TrainingArguments(
#     output_dir="./results/es",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss"
# )

# trainer_es = Trainer(
#     model=model_es,
#     args=training_args_es,
#     train_dataset=train_dataset_es,  
#     eval_dataset=val_dataset_es  # Now includes validation set
# )

# # Train Spanish Model
# trainer_es.train()


# Newer version

In [5]:
import json
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# === Load Tweets ===
with open("/kaggle/input/exist2025-all/EXIST2025_training_translated_en.json", "r", encoding="utf-8") as f:
    data_en = json.load(f)

with open("/kaggle/input/exist2025-all/EXIST2025_training_translated_es.json", "r", encoding="utf-8") as f:
    data_es = json.load(f)

# === Load gold_soft_train ===
with open("/kaggle/input/exist2025-all/EXIST2025_training_task1_3_gold_soft.json", "r", encoding="utf-8") as f:
    gold_soft = json.load(f)

# Convert gold_soft to a dict for fast access
gold_soft_dict = {entry["id"]: entry["value"] for entry in gold_soft}

# Define correct label classes in fixed order
CORRECT_LABELS = [
    "IDEOLOGICAL-INEQUALITY",
    "MISOGYNY-NON-SEXUAL-VIOLENCE",
    "OBJECTIFICATION",
    "SEXUAL-VIOLENCE",
    "STEREOTYPING-DOMINANCE",
    "NO"
]

# === Process Tweets with Corresponding Soft Labels ===
def process_data_with_soft_labels(data):
    tweets = []
    labels = []
    ids = []

    for entry in data.values():
        tweet_id = entry["id_EXIST"]
        tweet = entry["tweet"]

        if tweet_id not in gold_soft_dict:
            continue  # Skip if soft label not found

        soft_label_dict = gold_soft_dict[tweet_id]

        # Build soft label vector in fixed order
        soft_label_vector = [soft_label_dict.get(label, 0.0) for label in CORRECT_LABELS]

        tweets.append(tweet)
        labels.append(soft_label_vector)
        ids.append(tweet_id)

    return tweets, labels, ids

# Process both English and Spanish tweets
tweets_en, labels_en, ids_en = process_data_with_soft_labels(data_en)
tweets_es, labels_es, ids_es = process_data_with_soft_labels(data_es)

# === Tokenizer ===
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# === Custom Dataset Class ===
class TweetDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        tweet_id = self.ids[idx]
        labels = torch.tensor(self.labels[idx], dtype=torch.float)
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            "id": tweet_id,
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": labels
        }

# === Train-validation split ===
def get_datasets(tweets, labels, ids):
    train_texts, val_texts, train_labels, val_labels, train_ids, val_ids = train_test_split(
        tweets, labels, ids, test_size=0.2, random_state=42
    )
    train_dataset = TweetDataset(train_texts, train_labels, train_ids, tokenizer)
    val_dataset = TweetDataset(val_texts, val_labels, val_ids, tokenizer)
    return train_dataset, val_dataset

# === Create datasets ===
train_dataset_en, val_dataset_en = get_datasets(tweets_en, labels_en, ids_en)
train_dataset_es, val_dataset_es = get_datasets(tweets_es, labels_es, ids_es)

# === Train Model ===
def train_model(train_dataset, val_dataset, output_dir):
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-multilingual-cased",
        num_labels=len(CORRECT_LABELS),
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        logging_dir="./logs",
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()
    return trainer

# === Train English and Spanish models ===
trainer_en = train_model(train_dataset_en, val_dataset_en, output_dir="./results/en")
trainer_es = train_model(train_dataset_es, val_dataset_es, output_dir="./results/es")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.368000,0.375906
2,0.366900,0.365231
3,0.339000,0.362227
4,0.325700,0.364116


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert

Epoch,Training Loss,Validation Loss
1,0.371000,0.384112
2,0.367400,0.368844
3,0.344200,0.366302
4,0.333400,0.367768


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


# **Dev Testing starts from here**

In [6]:
with open("/kaggle/input/exist2025-all/EXIST2025_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Extract tweets and IDs
dev_tweets = [entry["tweet"] for entry in dev_data.values()]
dev_ids = [entry["id_EXIST"] for entry in dev_data.values()]


In [7]:
import json

# Load the dev dataset
with open("/kaggle/input/exist2025-all/EXIST2025_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# Split into English & Spanish
english_dev_tweets = []
english_dev_ids = []
spanish_dev_tweets = []
spanish_dev_ids = []

for entry in dev_data.values():
    tweet_id = entry["id_EXIST"]
    tweet = entry["tweet"]
    lang = entry["lang"]

    if lang == "en":
        english_dev_tweets.append(tweet)
        english_dev_ids.append(tweet_id)
    elif lang == "es":
        spanish_dev_tweets.append(tweet)
        spanish_dev_ids.append(tweet_id)

# Debugging: Check split sizes
print(f"English Dev Samples: {len(english_dev_tweets)}")
print(f"Spanish Dev Samples: {len(spanish_dev_tweets)}")


English Dev Samples: 489
Spanish Dev Samples: 549


In [8]:
import os
from transformers import BertForSequenceClassification

# Function to get the latest checkpoint
def get_latest_checkpoint(directory="./results"):
    checkpoints = [d for d in os.listdir(directory) if d.startswith("checkpoint-")]
    if not checkpoints:
        raise ValueError(f"No checkpoints found in {directory}")
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split('-')[-1]))[-1]
    return os.path.join(directory, latest_checkpoint)

# Load the best model checkpoint for English and Spanish
latest_checkpoint_en = get_latest_checkpoint("./results/en")
latest_checkpoint_es = get_latest_checkpoint("./results/es")

print(f"Using latest checkpoint for English: {latest_checkpoint_en}")
print(f"Using latest checkpoint for Spanish: {latest_checkpoint_es}")

# Load models
model_en = BertForSequenceClassification.from_pretrained(latest_checkpoint_en)
model_es = BertForSequenceClassification.from_pretrained(latest_checkpoint_es)


Using latest checkpoint for English: ./results/en/checkpoint-692
Using latest checkpoint for Spanish: ./results/es/checkpoint-692


# This is Soft Soft

In [12]:
def predict_on_dev(tweets, ids, model, tokenizer, label_classes, output_file):
    model.eval()
    results = []

    for tweet, tweet_id in zip(tweets, ids):
        encoding = tokenizer(tweet, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**encoding)

        logits = outputs.logits.squeeze()
        probs = torch.sigmoid(logits).cpu().numpy()

        # Convert probabilities to dictionary format and sort by highest probability
        soft_label_dict = {label_classes[i]: float(probs[i]) for i in range(len(label_classes))}
        sorted_soft_label_dict = dict(sorted(soft_label_dict.items(), key=lambda item: item[1], reverse=True))  # Sort descending

        results.append({
            "test_case": "EXIST2025",
            "id": tweet_id,
            "value": sorted_soft_label_dict  # Rename "soft_label" to "value" and sort it
        })

    # Save results
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print(f"Predictions saved to {output_file}")
    
label_classes = CORRECT_LABELS


# Run predictions
predict_on_dev(english_dev_tweets, english_dev_ids, model_en, tokenizer, label_classes, "EXIST2025_dev_predictions_en.json")
predict_on_dev(spanish_dev_tweets, spanish_dev_ids, model_es, tokenizer, label_classes, "EXIST2025_dev_predictions_es.json")


Predictions saved to EXIST2025_dev_predictions_en.json
Predictions saved to EXIST2025_dev_predictions_es.json


# This be Hard Hard

In [20]:
def predict_hard_labels_from_soft_model(tweets, ids, model, tokenizer, label_classes, output_file, threshold=0.33):
    """
    Uses the soft model to predict hard labels by applying a threshold.
    - Labels are assigned if their probability > threshold.
    - If no labels pass the threshold, assigns "NO".
    """
    model.eval()
    results = []

    for tweet, tweet_id in zip(tweets, ids):
        encoding = tokenizer(tweet, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**encoding)

        logits = outputs.logits.squeeze()
        probs = torch.sigmoid(logits).cpu().numpy()

        # Convert probabilities to hard labels using threshold
        hard_labels = [label_classes[i] for i, prob in enumerate(probs) if prob > threshold]

        # If no labels meet the threshold, assign "NO"
        if not hard_labels:
            hard_labels = ["NO"]

        results.append({
            "test_case": "EXIST2025",
            "id": tweet_id,
            "value": hard_labels  # Final hard labels
        })

    # Save results
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=4)

    print(f"Hard label predictions saved to {output_file}")

# Run hard label prediction using soft model
predict_hard_labels_from_soft_model(english_dev_tweets, english_dev_ids, model_en, tokenizer, label_classes, "EXIST2025_dev_predictions_hard_en.json")
predict_hard_labels_from_soft_model(spanish_dev_tweets, spanish_dev_ids, model_es, tokenizer, label_classes, "EXIST2025_dev_predictions_hard_es.json")


Hard label predictions saved to EXIST2025_dev_predictions_hard_en.json
Hard label predictions saved to EXIST2025_dev_predictions_hard_es.json


# Merging soft models ka dev set predictions

In [14]:
import json

# Load the Spanish predictions
with open("/kaggle/working/EXIST2025_dev_predictions_es.json", "r", encoding="utf-8") as f:
    es_data = json.load(f)

# Load the English predictions
with open("/kaggle/working/EXIST2025_dev_predictions_en.json", "r", encoding="utf-8") as f:
    en_data = json.load(f)

# Assuming both files contain lists of predictions, merge them
if isinstance(es_data, list) and isinstance(en_data, list):
    merged_data = es_data + en_data
else:
    raise ValueError("JSON structure is not a list. Ensure both files contain lists.")

# Save to a new file
output_filename = "EXIST2025_dev_predictions_merged_soft.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)

print(f"Merging complete! Saved to {output_filename}")


Merging complete! Saved to EXIST2025_dev_predictions_merged_soft.json


In [17]:
import json
import numpy as np

# Load your predictions file
with open('EXIST2025_dev_predictions_merged_soft.json', 'r') as f:
    predictions = json.load(f)

# Define the snapping values (multiples of 1/6)
snap_vals = np.array([i / 6 for i in range(7)])  # [0.0, 0.1667, ..., 1.0]

def snap_to_nearest_sixth(value):
    return float(snap_vals[np.argmin(np.abs(snap_vals - value))])

# Snap each value in the 'value' dict
for entry in predictions:
    entry['value'] = {k: snap_to_nearest_sixth(v) for k, v in entry['value'].items()}

# Save the snapped predictions to a new file
with open('EXIST2025_dev_predictions_snapped_soft.json', 'w') as f:
    json.dump(predictions, f, indent=2)


# Merging hard models ka dev set predictions

In [21]:
import json

# Load the Spanish predictions
with open("/kaggle/working/EXIST2025_dev_predictions_hard_es.json", "r", encoding="utf-8") as f:
    es_data = json.load(f)

# Load the English predictions
with open("/kaggle/working/EXIST2025_dev_predictions_hard_en.json", "r", encoding="utf-8") as f:
    en_data = json.load(f)

# Assuming both files contain lists of predictions, merge them
if isinstance(es_data, list) and isinstance(en_data, list):
    merged_data = es_data + en_data
else:
    raise ValueError("JSON structure is not a list. Ensure both files contain lists.")

# Save to a new file
output_filename = "EXIST2025_dev_predictions_merged_hard.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(merged_data, f, indent=4, ensure_ascii=False)

print(f"Merging complete! Saved to {output_filename}")


Merging complete! Saved to EXIST2025_dev_predictions_merged_hard.json


# Metric Calculation

In [15]:
pip install pyevall

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 19.6 MB/s eta 0:00:0000:01
  Created wheel for pyevall: filename=PyEvALL-0.1.78-py3-none-any.whl size=34777 sha256=b6509b169da725acfcc9b3e75fecd5cafff970bb0b6e201f0aa38769348154be
  Stored in directory: /root/.cache/pip/wheels/f0/3a/51/f8c268e67356c15a602eef8ac7a5e18ba4677b4ec8b45b8a25
  Created wheel for jsbeautifier: filename=jsbeautifier-1.14.9-py3-none-any.whl size=94157 sha256=53111ec2c7e0f439f2be4ae9c13588a5aa3756bb5f783e58680ed6048e72d6b0
  Stored in directory: /root/.cache/pip/wheels/c4/5c/25/09f8b2e8dddb4fc3d70817c67b375a9069a2628847ffbdfc65
Successfully built pyevall jsbeautifier
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
  

# This is hard metrics

In [22]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

predictions = "/kaggle/working/EXIST2025_dev_predictions_merged_hard.json"         
gold = "/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_gold_hard.json" 
test = PyEvALLEvaluation() 
params= dict() 
params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED  
# metrics=["ICMSoft", "ICMSoftNorm", "CrossEntropy"]     # for soft    
metrics=["ICM", "ICMNorm" ,"FMeasure"] 
TASK1_3_HIERARCHY = {"YES":["IDEOLOGICAL-INEQUALITY","STEREOTYPING-DOMINANCE","OBJECTIFICATION", "SEXUAL-VIOLENCE", "MISOGYNY-NON-SEXUAL-VIOLENCE"], "NO":[]}
params[PyEvALLUtils.PARAM_HIERARCHY]= TASK1_3_HIERARCHY  
report= test.evaluate(predictions, gold, metrics, **params) 
report.print_report()



2025-04-14 01:30:54,227 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-04-14 01:30:54,406 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-04-14 01:30:54,954 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-04-14 01:30:54,958 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-04-14 01:30:55,488 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-04-14 01:30:56,002 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": -0.30018

# These are soft metrics

In [19]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Define file paths
predictions = "/kaggle/working/EXIST2025_dev_predictions_snapped_soft.json"  # Change to your actual prediction file
gold = "/kaggle/input/exist2025-all/EXIST2025_dev_task1_3_gold_soft.json"   # Change to your actual gold file

# Define hierarchical structure for subtask 1.3
TASK1_3_HIERARCHY = {
    "YES": ["IDEOLOGICAL-INEQUALITY", "STEREOTYPING-DOMINANCE",
            "OBJECTIFICATION", "SEXUAL-VIOLENCE", "MISOGYNY-NON-SEXUAL-VIOLENCE"],
    "NO": []
}

# Initialize PyEvALL evaluation
evaluator = PyEvALLEvaluation()

# Set evaluation parameters
params = dict()
params[PyEvALLUtils.PARAM_HIERARCHY] = TASK1_3_HIERARCHY
params[PyEvALLUtils.PARAM_REPORT] = PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED  # Embedded report

# Define evaluation metrics
# metrics = ["ICM", "ICMNorm", "FMeasure"]
metrics=["ICMSoft", "ICMSoftNorm"]

# Run evaluation
report = evaluator.evaluate(predictions, gold, metrics, **params)

# Print evaluation report
report.print_report()


2025-04-14 01:17:36,335 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICMSoft', 'ICMSoftNorm']
2025-04-14 01:17:36,685 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-04-14 01:17:38,174 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM-Soft Normalized evaluation method
2025-04-14 01:17:38,177 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
2025-04-14 01:17:39,617 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Soft evaluation method
{
  "metrics": {
    "ICMSoft": {
      "name": "Information Contrast Model Soft",
      "acronym": "ICM-Soft",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": -2.588298008906674
        }],
        "average_per_test_case": -2.588298008906674
      }
    },
 